In [ ]:
!pip install ipynb --upgrade

In [1]:
from ipynb.fs.full.NER import get_entities
import pandas as pd 

### Load csv

In [2]:
data = pd.read_csv('data_files/wallstreetbets_26.csv')

----

#### Functions

In [6]:
# Returns list with submissions dataframe and comments dataframe
def transform_reddit_data(data: pd.DataFrame):
        dfs = []
        # submissions
        df_submissions = data.drop(['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title'],axis=1)
        df_submissions['created'] = df_submissions['created'].apply(datetime_to_date)
        df_submissions = df_submissions.dropna()
        df_submissions.rename(columns = {'post_id':'id'}, inplace = True)
        df_submissions['Organizations'] = df_submissions['text'].apply(get_entities).apply(clean_orgs)
        dfs.append(df_submissions)
        # comments
        df_comments = data.drop(['Unnamed: 0', 'fullname', 'post_id', 'title'],axis=1)
        df_comments['created'] = df_comments['created'].apply(datetime_to_date)
        df_comments = df_comments.dropna()
        df_comments['Organizations'] = df_comments['text'].apply(get_entities).apply(clean_orgs)
        dfs.append(df_comments)
        
        return dfs

In [4]:
# Changes utc timestamp to datetime.date
def datetime_to_date(timestamp):
    return pd.to_datetime(timestamp).date()

In [5]:
# Mentions that we are interested in
selected_orgs =  ['AMC', 'Palantir Technologies', 'PLTR', 'FORD', 'Lordstown Motors', 'RIDE', 'Virgin Galactic', 'SPCE', 'AI', 'C3.AI', 'TSLA', 'GE', 'GME', 'AAPL', 'Tesla', 'Apple', 'General Electric', 'GE', 'NOK', 'Nokia']

def clean_orgs(organizations):
    orgs = []
    for org in organizations:
        if org in selected_orgs:
            orgs.append(org)
    return orgs

## A Look at the mentions

In [ ]:
# Creates a list of mentionend entities
data['Organizations'] = data['text'].apply(get_entities)
orgs = data['Organizations'].to_list()
orgs_flat = [org for sublist in orgs for org in sublist] # Pulls out entities from the nested lists in orgs => new flat list
# Print 20 most mentions ORGs
from collections import Counter
org_freq = Counter(orgs_flat)
org_freq.most_common(20)                                                       

## Save data to database

In [11]:
dfs = transform_reddit_data(data)
submissions = dfs[0]
comments = dfs[1]

In [17]:
# SUBMISSIONS subset
df = submissions[:3]


In [18]:
from neo4j import GraphDatabase

In [ ]:
for index, row in df.iterrows():
    tx.evaluate('''
       MERGE (sm:submission {property:$Person1})
       MERGE (b:person2 {property:$Person2})
       MERGE (a)-[r:R_TYPE]->(b)
       ''', parameters = {'Person1': int(row['Person1']), 'Person2': int(row['Person2'])})
tx.commit()


 CREATE (sm:submission:'''+subreddit+'''{date:$date, id:$id, text:$text})
  